<h1 align="center">Scientific Programming in Python</h1>
<h2 align="center">Topic 5: Accelerating Python with Cython: Writting C in Python </h2> 


_Notebook created by Martín Villanueva - `martin.villanueva@usm.cl` - DI UTFSM - May2017._

In [1]:
%matplotlib inline

import numpy as np
import numexpr as ne
import numba
import math
import random
import matplotlib.pyplot as plt
import scipy as sp
import sys

%load_ext Cython

## Table of Contents
* [1.- Cython Basic Usage](#cython)
* [2.- Advanced usage](#cython++)
* [3.- Pure C in Python](#C)


<div id='cython' />
## 1.- Cython Basic Usage

__Cython__ is both a __Superset of Python__ and a __Python Library__ that lets you combine C and Python in various ways. There are two main use-cases:
1. Optimizing your Python code by statically compiling it to C.
2. Wrapping a C/C++ library in Python.

In order to get it properly working, you need Cython and a C compiler:
1. __Cython__: `conda install cython`
2. __C compiler__: Install GNU C compiler with your package manager (Unix/Linux) or install Xcode (OSX).

***

We will introduce the basic Cython usage by impementing the  [__Eratosthenes Sieve Algorithm__](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes), which is an algorithm to find all prime numbers smaller than a given number.

In [8]:
def primes_python(n):
    primes = [False, False] + [True] * (n - 2)
    i= 2
    while i < n:
        # We do not deal with composite numbers.
        if not primes[i]:
            i += 1
            continue 
        k= i+i
        # We mark multiples of i as composite numbers.
        while k < n:
            primes[k] = False
            k += i 
        i += 1
    # We return all numbers marked with True.
    return [i for i in range(2, n) if primes[i]]

In [9]:
primes_python(20)

[2, 3, 5, 7, 11, 13, 17, 19]

Let's evaluate the performance for the first version:

In [10]:
tp = %timeit -o primes_python(10000)

100 loops, best of 3: 5.11 ms per loop


And now we write our first Cython version, by just adding `%%cython` magic in the first line of the cell:

In [11]:
%%cython
def primes_cython1(n):
    primes = [False, False] + [True] * (n - 2)
    i= 2
    while i < n:
        # We do not deal with composite numbers.
        if not primes[i]:
            i += 1
            continue 
        k= i+i
        # We mark multiples of i as composite numbers.
        while k < n:
            primes[k] = False
            k += i 
        i += 1
    # We return all numbers marked with True.
    return [i for i in range(2, n) if primes[i]]

In [13]:
tc1 = %timeit -o primes_cython1(10000)

100 loops, best of 3: 2.61 ms per loop


__We achieve x2 speed improvement doing (practically) nothing!__.

When we add `%%cython` at the beginning of the cell, the code gets compiled by Cython into a C extension. Then, this extension is loaded, and the compiled function is readily available in the interactive namespace. 

Lets help the compiler by explicitly defining the type of the variables with the __`cdef`__ macro/keyword:

In [14]:
%%cython
def primes_cython2(int n):
    # Note the type declarations below
    cdef list primes = [False, False] + [True] * (n - 2)
    cdef int i = 2
    cdef int k = 0
    # The rest of the functions is unchanged
    while i < n:
        # We do not deal with composite numbers.
        if not primes[i]:
            i += 1
            continue 
        k= i+i
        # We mark multiples of i as composite numbers.
        while k < n:
            primes[k] = False
            k += i 
        i += 1
    # We return all numbers marked with True.
    return [i for i in range(2, n) if primes[i]]

In [15]:
tc2 = %timeit -o primes_cython2(10000)

1000 loops, best of 3: 308 µs per loop


In [18]:
print("Cython version 1 speedup: {0}".format(tp.best/tc1.best))
print("Cython version 2 speedup: {0}".format(tp.best/tc2.best))

Cython version 1 speedup: 1.9537728803827772
Cython version 2 speedup: 16.57108263494032


__Then__: _In general, Cython will be the most efficient when it can compile data structures and operations directly to C by __making as few CPython API calls as possible__. Specifying the types of the variables often leads to greater speed improvements._

Just for curiosity let's see the performance Numba's JIT achieves:

In [22]:
@numba.jit(nopython=True)
def primes_numba(n):
    primes = [False, False] + [True] * (n - 2)
    i= 2
    while i < n:
        # We do not deal with composite numbers.
        if not primes[i]:
            i += 1
            continue 
        k= i+i
        # We mark multiples of i as composite numbers.
        while k < n:
            primes[k] = False
            k += i 
        i += 1
    # We return all numbers marked with True.
    res = []
    for i in range(2,n):
        if primes[i]: res.append(i)
    return res

In [42]:
tn = %timeit -o primes_numba(10000)

10000 loops, best of 3: 159 µs per loop


Numba wins this time! but: __This is not the final form of Cython...__ 

### Inspecting Cython bottlenecks with annotations

We can inspect the C code generated by Cython with the `-a` argument. Let's inspect the code used above.

The non-optimized lines will be shown in a gradient of yellow (__white lines are faster, yellow lines are slower__), telling you which lines are the least efficiently compiled to C. By clicking on a line, you can see the generated C code corresponding to that line.

In [43]:
%%cython -a
def primes_cython1(n):
    primes = [False, False] + [True] * (n - 2)
    i= 2
    while i < n:
        # We do not deal with composite numbers.
        if not primes[i]:
            i += 1
            continue 
        k= i+i
        # We mark multiples of i as composite numbers.
        while k < n:
            primes[k] = False
            k += i 
        i += 1
    # We return all numbers marked with True.
    return [i for i in range(2, n) if primes[i]]

In [44]:
%%cython -a
def primes_cython2(int n):
    # Note the type declarations below
    cdef list primes = [False, False] + [True] * (n - 2)
    cdef int i = 2
    cdef int k = 0
    # The rest of the functions is unchanged
    while i < n:
        # We do not deal with composite numbers.
        if not primes[i]:
            i += 1
            continue 
        k= i+i
        # We mark multiples of i as composite numbers.
        while k < n:
            primes[k] = False
            k += i 
        i += 1
    # We return all numbers marked with True.
    return [i for i in range(2, n) if primes[i]]

### Alternative usage of Cython: Outside the notebook

If you want to use Cython outside the notebook (the way it was thought...), you have to do the work of the magic:
1. Write the function into a `.pyx` file.
2. __Cythonize it__ with `cython filename.pyx` generating the `filename.c` file.
3. Compile it with GCC: 

`gcc -shared -fPIC -fwrapv -O3 -fno-strict-aliasing -I/home/mavillan/anaconda3/include/python3.6m -o primes.so primes.c`

<div id='cython++' />
## 2.- Advanced usage

In this section we will consider the example of computing a distance matrix: Given the matrices $A_{m,3}$ and $B_{n,3}$ (each row is a 3D-position), the distance matrix has entries $D_{i,j} = d(A[i],B[j])$.

### NumPy Arrays

You can use NumPy from Cython exactly the same as in regular Python, but by doing so you are losing potentially high speedups because Cython has support for fast access to NumPy arrays.

In [66]:
# Matrices to use
A = np.random.random((1000,3))
B = np.random.random((500,3))

In [59]:
def dist(a, b):
    return np.sqrt(np.sum((a-b)**2))

def distance_matrix_python(A, B):
    m = A.shape[0]
    n = B.shape[0]
    D = np.empty((m,n))
    for i in range(m):
        for j in range(n):
            D[i,j] = dist(A[i],B[j])
    return D

In [60]:
%timeit distance_matrix_python(A,B)

1 loop, best of 3: 1min 50s per loop


In [62]:
%%cython -a
import numpy as np

def dist(a, b):
    return np.sqrt(np.sum((a-b)**2))

def distance_matrix_cython0(A, B):
    m = A.shape[0]
    n = B.shape[0]
    D = np.empty((m,n))
    for i in range(m):
        for j in range(n):
            D[i,j] = dist(A[i],B[j])
    return D

In [63]:
%timeit distance_matrix_cython0(A,B)

1 loop, best of 3: 1min 51s per loop


Now let's improve this naive Cython implementation by statically defining the types of the variables:

In [118]:
%%cython -a
import numpy as np
cimport numpy as cnp

ctypedef cnp.float64_t float64_t

def dist(cnp.ndarray[float64_t, ndim=1] a, cnp.ndarray[float64_t, ndim=1] b):
    return np.sqrt(np.sum((a-b)**2))

def distance_matrix_cython1(cnp.ndarray[float64_t, ndim=2] A, cnp.ndarray[float64_t, ndim=2] B):
    cdef:
        int m = A.shape[0]
        int n = B.shape[0]
        int i,j
        cnp.ndarray[float64_t, ndim=2] D = np.empty((m,n))
    for i in range(m):
        for j in range(n):
            D[i,j] = dist(A[i], B[j])
    return D

In [120]:
%timeit -n 10 distance_matrix_cython1(A,B)

10 loops, best of 3: 5.34 s per loop


In [121]:
%%cython -a
import numpy as np
cimport numpy as cnp

ctypedef cnp.float64_t float64_t
from libc.math cimport sqrt

def dist(cnp.ndarray[float64_t, ndim=1] a, cnp.ndarray[float64_t, ndim=1] b):
    cdef:
        int i = 0
        int n = a.shape[0]
        float ret = 0
    for i in range(n):
        ret += (a[i]-b[i])**2
    return sqrt(ret)


def distance_matrix_cython2(cnp.ndarray[float64_t, ndim=2] A, cnp.ndarray[float64_t, ndim=2] B):
    cdef:
        int m = A.shape[0]
        int n = B.shape[0]
        int i,j
        cnp.ndarray[float64_t, ndim=2] D = np.empty((m,n))
    for i in range(m):
        for j in range(n):
            D[i,j] = dist(A[i], B[j])
    return D

In [122]:
%timeit -n 10 distance_matrix_cython2(A,B)

10 loops, best of 3: 1.17 s per loop


### Typed Memory Views

__Typed memoryviews__ allow efficient access to memory buffers, such as those underlying NumPy arrays, __without incurring any Python overhead__. Memoryviews are similar to the current NumPy array buffer support (np.ndarray[np.float64_t, ndim=2]), but they have more features and cleaner syntax.

They can handle a wider variety of sources of array data. For example, they can handle C arrays and the Cython array type (Cython arrays).

__Syntaxis:__ `dtype[:,::1]` where `::1` indicates the axis where elements are contiguous. 

In [124]:
%%cython -a
import numpy as np
cimport numpy as cnp

ctypedef cnp.float64_t float64_t
from libc.math cimport sqrt

def dist(float64_t[::1] a, float64_t[::1] b):
    cdef:
        int i = 0
        int n = a.shape[0]
        float ret = 0
    for i in range(n):
        ret += (a[i]-b[i])**2
    return sqrt(ret)


def distance_matrix_cython3(float64_t[:,::1] A, float64_t[:,::1] B):
    cdef:
        int m = A.shape[0]
        int n = B.shape[0]
        int i,j
        float64_t[:,::1] D = np.empty((m,n))
    for i in range(m):
        for j in range(n):
            D[i,j] = dist(A[i], B[j])
    return D

In [125]:
%timeit -n 10 distance_matrix_cython3(A,B)

10 loops, best of 3: 379 ms per loop


### Compiler optimization

With the `-c` option we can pass the compiler (`gcc`) optimization options. Below we use the most common of them:

In [126]:
%%cython -a -c=-fPIC -c=-fwrapv -c=-O3 -c=-fno-strict-aliasing
import numpy as np
cimport numpy as cnp

ctypedef cnp.float64_t float64_t
from libc.math cimport sqrt

def dist(float64_t[::1] a, float64_t[::1] b):
    cdef:
        int i = 0
        int n = a.shape[0]
        float ret = 0
    for i in range(n):
        ret += (a[i]-b[i])**2
    return sqrt(ret)


def distance_matrix_cython4(float64_t[:,::1] A, float64_t[:,::1] B):
    cdef:
        int m = A.shape[0]
        int n = B.shape[0]
        int i,j
        float64_t[:,::1] D = np.empty((m,n))
    for i in range(m):
        for j in range(n):
            D[i,j] = dist(A[i], B[j])
    return D

In [127]:
%timeit -n 10 distance_matrix_cython4(A,B)

10 loops, best of 3: 409 ms per loop


### Compiler directives

__ Compiler directives are instructions which affect the behavior of Cython code. __


`cdivision` (True / False)
* If set to False, Cython will adjust the remainder and quotient operators C types to match those of Python ints (which differ when the operands have opposite signs) and raise a ZeroDivisionError when the right operand is 0. This has up to a 35% speed penalty. If set to True.

`boundscheck` (True / False)
* If set to False, Cython is free to assume that indexing operations ([]-operator) in the code will not cause any IndexErrors to be raised. Lists, tuples, and strings are affected only if the index can be determined to be non-negative (or if wraparound is False). Conditions which would normally trigger an IndexError may instead cause segfaults or data corruption if this is set to False. Default is True.

`nonecheck` (True / False)
* If set to False, Cython is free to assume that native field accesses on variables typed as an extension type, or buffer accesses on a buffer variable, never occurs when the variable is set to None. Otherwise a check is inserted and the appropriate exception is raised. This is off by default for performance reasons. Default is False.

`wraparound` (True / False)
* In Python arrays can be indexed relative to the end. For example A[-1] indexes the last value of a list. In C negative indexing is not supported. If set to False, Cython will neither check for nor correctly handle negative indices, possibly causing segfaults or data corruption. Default is True.

`initializedcheck` (True / False)
* If set to True, Cython checks that a memoryview is initialized whenever its elements are accessed or assigned to. Setting this to False disables these checks. Default is True.

For all the compilation directives see [here](http://cython.readthedocs.io/en/latest/src/reference/compilation.html#compiler-directives).

In [136]:
%%cython -a -c=-fPIC -c=-fwrapv -c=-O3 -c=-fno-strict-aliasing
#!python
#cython: cdivision=True, boundscheck=False, nonecheck=False, wraparound=False, initializedcheck=False

import numpy as np
cimport numpy as cnp

ctypedef cnp.float64_t float64_t
from libc.math cimport sqrt

def dist(float64_t[::1] a, float64_t[::1] b):
    cdef:
        int i = 0
        int n = a.shape[0]
        float ret = 0
    for i in range(n):
        ret += (a[i]-b[i])**2
    return sqrt(ret)


def distance_matrix_cython5(float64_t[:,::1] A, float64_t[:,::1] B):
    cdef:
        int m = A.shape[0]
        int n = B.shape[0]
        int i,j
        float64_t[:,::1] D = np.empty((m,n))
    for i in range(m):
        for j in range(n):
            D[i,j] = dist(A[i,:], B[j,:])
    return D

In [129]:
%timeit -n 10 distance_matrix_cython5(A,B)

10 loops, best of 3: 369 ms per loop


### Pure C functions

With the `cdef` keywork we can realy define C function, as we shown below. In such functions all variable types should be defined and __should have a return type__, and can't be called directly in Python, i.e, only can be called by functions defined in the same module.

There is a midpoint between `def` and `cdef` which automatically creates a Python function with the same name, so the function can be called directly. 

In [130]:
%%cython -a -c=-fPIC -c=-fwrapv -c=-O3 -c=-fno-strict-aliasing
#!python
#cython: cdivision=True, boundscheck=False, nonecheck=False, wraparound=False, initializedcheck=False

import numpy as np
cimport numpy as cnp

ctypedef cnp.float64_t float64_t
from libc.math cimport sqrt

cdef float64_t dist(float64_t[::1] a, float64_t[::1] b):
    cdef:
        int i = 0
        int n = a.shape[0]
        float ret = 0
    for i in range(n):
        ret += (a[i]-b[i])**2
    return sqrt(ret)


def distance_matrix_cython6(float64_t[:,::1] A, float64_t[:,::1] B):
    cdef:
        int m = A.shape[0]
        int n = B.shape[0]
        int i,j
        float64_t[:,::1] D = np.empty((m,n))
    for i in range(m):
        for j in range(n):
            D[i,j] = dist(A[i,:], B[j,:])
    return D

In [131]:
%timeit -n 10 distance_matrix_cython6(A,B)

10 loops, best of 3: 17.4 ms per loop


***
#### Example of `cdef` and `cpdef`

In [139]:
%%cython -a -c=-fPIC -c=-fwrapv -c=-O3 -c=-fno-strict-aliasing
#!python
#cython: cdivision=True, boundscheck=False, nonecheck=False, wraparound=False, initializedcheck=False

cimport numpy as cnp
ctypedef cnp.float64_t float64_t

cdef float64_t test1(float64_t a, float64_t b):
    return a+b

In [142]:
%%cython -a -c=-fPIC -c=-fwrapv -c=-O3 -c=-fno-strict-aliasing
#!python
#cython: cdivision=True, boundscheck=False, nonecheck=False, wraparound=False, initializedcheck=False

cimport numpy as cnp
ctypedef cnp.float64_t float64_t

cpdef float64_t test2(float64_t a, float64_t b):
    return a+b

In [143]:
test2(1,1)

2.0

***
### Function inlining

In computing, inline expansion, or inlining, is a manual or compiler optimization that __replaces a function call site with the body of the called function__. 

__As a rule of thumb__: Some inlining will improve speed at very minor cost of space, but excess inlining will hurt speed, due to __inlined code consuming too much of the instruction cache__, and also cost significant space.

In [132]:
%%cython -a -c=-fPIC -c=-fwrapv -c=-O3 -c=-fno-strict-aliasing
#!python
#cython: cdivision=True, boundscheck=False, nonecheck=False, wraparound=False, initializedcheck=False

import numpy as np
cimport numpy as cnp

ctypedef cnp.float64_t float64_t
from libc.math cimport sqrt

cdef inline float64_t dist(float64_t[::1] a, float64_t[::1] b):
    cdef:
        int i = 0
        int n = a.shape[0]
        float ret = 0
    for i in range(n):
        ret += (a[i]-b[i])**2
    return sqrt(ret)

def distance_matrix_cython7(float64_t[:,::1] A, float64_t[:,::1] B):
    cdef:
        int m = A.shape[0]
        int n = B.shape[0]
        int i,j
        float64_t[:,::1] D = np.empty((m,n))
    for i in range(m):
        for j in range(n):
            D[i,j] = dist(A[i,:], B[j,:])
    return D

In [133]:
%timeit -n 10 distance_matrix_cython7(A,B)

10 loops, best of 3: 18 ms per loop


### What about Numba?

In [134]:
@numba.jit(nopython=True)
def dist(a, b):
    n = a.shape[0]
    ret = 0
    for i in range(n):
        ret += (a[i]-b[i])**2
    return math.sqrt(ret)

@numba.jit(nopython=True)
def distance_matrix_numba(A, B):
    m = A.shape[0]
    n = B.shape[0]
    D = np.empty((m,n))
    for i in range(m):
        for j in range(n):
            D[i,j] = dist(A[i,:], B[j,:])
    return D

In [135]:
%timeit -n 10 distance_matrix_numba(A,B)

10 loops, best of 3: 38 ms per loop


## 4.- Other advanced things you can do with Cython

We have seen that with Cython we can implement our algorithms achieving C performance. Moreover it is very versatile and we can do some other advanced thing with it:

### Object-oriented programming: Classes and methods

To support object-oriented programming, Cython supports writing normal Python classes exactly as in Python.

In [144]:
%%cython -a -c=-fPIC -c=-fwrapv -c=-O3 -c=-fno-strict-aliasing
#!python
#cython: cdivision=True, boundscheck=False, nonecheck=False, wraparound=False, initializedcheck=False

cdef class A(object):
    def   d(self): return 0
    cdef  int c(self): return 0
    cpdef int p(self): return 0

    def test_def(self, long num):
        while num > 0:
            self.d()
            num -= 1

    def test_cdef(self, long num):
        while num > 0:
            self.c()
            num -= 1

    def test_cpdef(self, long num):
        while num > 0:
            self.p()
            num -= 1

In [148]:
%%timeit n = 1000000
a1 = A()
a1.test_def(n)

10 loops, best of 3: 36.7 ms per loop


In [149]:
%%timeit n = 1000000
a1 = A()
a1.test_cdef(n)

100 loops, best of 3: 1.91 ms per loop


In [150]:
%%timeit n = 1000000
a1 = A()
a1.test_cpdef(n)

100 loops, best of 3: 3.12 ms per loop


### C library Wrapping
Wrapper libraries (or library wrappers) consist of a thin layer of code which translates a library's existing interface into a compatible interface. Cython allows us to do this with C libraries... In fact many important project use Cython to do that:
* __Scikit-Learn__ use Cython to wrap many machine learning routines written in C (LibSVM).
* __OpenCV__ for python.
* __Scikit-Image__.
* __SciPy__ Wraps BLAS, LAPACK and others.
* Etc.